#### OBJETIVO NOTEBOOK
- Preprocesamiento para eliminar columnas o filas duplicadas, enriquecer la base de datos con información del sector.

#### INSTRUCCIONES
- El fichero final se ha guardado en la carpeta data hist con el siguiente nombre df_sp500_11y_withsectors.csv. El fichero original tiene como separador de decimales '.'. Para guardarlo lo he convertido a ',' ya que si no me daba problemas de lectura el power bi.

# LIBRERIAS

In [ ]:
# cálculos y dataframes
import numpy as np
import pandas as pd
import math

# fechas
from datetime import datetime

# IMPORTAR DATA

In [ ]:
# conexion a drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_sp500 = pd.read_csv("/content/drive/MyDrive/ACADEMIC/MASTER DATA SCIENCE & AI (NUCLIO DIGITAL SCHOOL)/TFM/TFM SP500 COMPARTIDO/DATA_HIST/sp500_financials_and_prices_141224_finalToReview.csv")

In [ ]:
df_sp500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5485 entries, 0 to 5484
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   fiscalDateEnding                       5485 non-null   object 
 1   totalRevenue                           5342 non-null   float64
 2   depreciation                           3483 non-null   float64
 3   depreciationAndAmortization            5332 non-null   float64
 4   ebitda                                 5332 non-null   float64
 5   netIncome                              5342 non-null   float64
 6   ticker_income                          5342 non-null   object 
 7   cashAndCashEquivalentsAtCarryingValue  5459 non-null   float64
 8   inventory                              5015 non-null   float64
 9   currentNetReceivables                  4430 non-null   float64
 10  totalLiabilities                       5461 non-null   float64
 11  curr

# DATA CLEANING & EDA

In [ ]:
df_sp500.isnull().sum()

,0
fiscalDateEnding,0
totalRevenue,143
depreciation,2002
depreciationAndAmortization,153
ebitda,153
netIncome,143
ticker_income,143
cashAndCashEquivalentsAtCarryingValue,26
inventory,470
currentNetReceivables,1055


In [ ]:
df_sp500.nunique()

,0
fiscalDateEnding,298
totalRevenue,5251
depreciation,2475
depreciationAndAmortization,3644
ebitda,4980
netIncome,4831
ticker_income,501
cashAndCashEquivalentsAtCarryingValue,4845
inventory,4398
currentNetReceivables,4055


Existen tres columnas categóricas que representan la misma información: ticker. Esto se debe a que la base de datos se ha construido a partir de la unión de tres tablas distintas (income_statement, balance_sheet y cashflow), complementadas con una cuarta tabla para obtener el último precio del año.

Para garantizar la coherencia de los datos, unificaremos estas tres columnas en una sola, asegurando que cada fila tenga un único valor de ticker, evitando inconsistencias en la información.

In [ ]:
df_sp500[["ticker_income", "ticker_balance", "ticker_cashflow"]]

,ticker_income,ticker_balance,ticker_cashflow
0,MMM,MMM,MMM
1,MMM,MMM,MMM
2,MMM,MMM,MMM
3,MMM,MMM,MMM
4,MMM,MMM,MMM
...,...,...,...
5480,ZTS,ZTS,ZTS
5481,ZTS,ZTS,ZTS
5482,ZTS,ZTS,ZTS
5483,ZTS,ZTS,ZTS


In [ ]:
df_sp500[df_sp500["ticker_balance"].isnull()][["ticker_income","ticker_cashflow"]]

,ticker_income,ticker_cashflow
105,A,A
254,AMTM,AMTM
463,ADM,ADM
1175,CSCO,CSCO
1515,DECK,DECK
2500,HPE,HPE
3114,LDOS,LDOS
3433,MCHP,MCHP
3434,MCHP,MCHP


In [ ]:
# rellenar los valores nulos de la columna ticker_balance con el valor de la columna ticker_income, si este es nulo, rellenar con el valor de la columna ticker_cashflow
df_sp500["ticker_balance"] = (
    df_sp500["ticker_balance"]
    .fillna(df_sp500["ticker_income"])
    .fillna(df_sp500["ticker_cashflow"])
)

# comprobamos que ya no tengamos nulos
df_sp500[df_sp500["ticker_balance"].isnull()]

,fiscalDateEnding,totalRevenue,depreciation,depreciationAndAmortization,ebitda,netIncome,ticker_income,cashAndCashEquivalentsAtCarryingValue,inventory,currentNetReceivables,...,ticker_balance,capitalExpenditures,cashflowFromInvestment,dividendPayout,cashflowFromFinancing,operatingCashflow,ticker_cashflow,year,date,Year-End Price


In [ ]:
# eliminar las columna ticker_income y ticker cashflow, y renombrar la columna ticker balance
df_sp500 = (
    df_sp500
    .drop(columns= ["ticker_income", "ticker_cashflow"])
    .rename(columns={"ticker_balance":"ticker"})
)

In [ ]:
df_sp500.isnull().sum()

,0
fiscalDateEnding,0
totalRevenue,143
depreciation,2002
depreciationAndAmortization,153
ebitda,153
netIncome,143
cashAndCashEquivalentsAtCarryingValue,26
inventory,470
currentNetReceivables,1055
totalLiabilities,24


In [ ]:
# insertar la columna ticker al inicio del dataframe
ticker_col = df_sp500.pop("ticker")
df_sp500.insert(0, "ticker", ticker_col)

In [ ]:
# insertar la columna year en la segunda posición
year_col = df_sp500.pop("year")
df_sp500.insert(1, "year", year_col)

In [ ]:
df_sp500[df_sp500["ticker"]=='AAPL']

,ticker,year,fiscalDateEnding,totalRevenue,depreciation,depreciationAndAmortization,ebitda,netIncome,cashAndCashEquivalentsAtCarryingValue,inventory,...,totalShareholderEquity,commonStockSharesOutstanding,shortLongTermDebtTotal,capitalExpenditures,cashflowFromInvestment,dividendPayout,cashflowFromFinancing,operatingCashflow,date,Year-End Price
417,AAPL,2013,2013-09-30,1.707740e+11,NaN,5.800000e+09,5.609100e+10,3.703700e+10,1.425900e+10,1.764000e+09,...,1.235490e+11,8.992130e+08,1.696000e+10,8.165000e+09,-3.377400e+10,1.052800e+10,-1.637900e+10,5.366600e+10,2013-12-31,17.479847
418,AAPL,2014,2014-09-30,1.824110e+11,NaN,6.900000e+09,6.076700e+10,3.951000e+10,1.384400e+10,2.111000e+09,...,1.115470e+11,5.866161e+09,3.529500e+10,9.571000e+09,-2.257900e+10,1.103100e+10,-3.754900e+10,1.091330e+11,2014-12-31,24.579730
419,AAPL,2015,2015-09-30,2.329820e+11,NaN,9.200000e+09,8.244800e+10,5.339400e+10,2.112000e+10,2.349000e+09,...,1.193550e+11,5.578753e+09,1.197870e+11,1.148800e+10,-5.627400e+10,1.156100e+10,-1.771600e+10,8.126600e+10,2015-12-31,23.837929
420,AAPL,2016,2016-09-30,2.156390e+11,8.300000e+09,8.300000e+09,7.112800e+10,4.568700e+10,2.048400e+10,2.132000e+09,...,1.282490e+11,5.336166e+09,1.646940e+11,1.354800e+10,-4.597700e+10,1.215000e+10,-2.089000e+10,6.623100e+10,2016-12-30,26.813110
421,AAPL,2017,2017-09-30,2.292340e+11,8.200000e+09,1.200000e+09,6.761200e+10,4.835100e+10,2.028900e+10,4.855000e+09,...,1.340470e+11,5.126201e+09,2.197940e+11,1.279500e+10,-4.644600e+10,1.276900e+10,-1.797400e+10,6.422500e+10,2017-12-29,39.810862
422,AAPL,2018,2018-09-30,2.655950e+11,9.300000e+09,9.300000e+09,8.180100e+10,5.953100e+10,2.591300e+10,3.956000e+09,...,1.071470e+11,4.754986e+09,2.201320e+11,1.331300e+10,1.606600e+10,1.371200e+10,-8.787600e+10,7.743400e+10,2018-12-31,37.664536
423,AAPL,2019,2019-09-30,2.565980e+11,1.130000e+10,1.130000e+10,7.647700e+10,5.525600e+10,4.884400e+10,4.106000e+09,...,9.048800e+10,4.443236e+09,2.091350e+11,1.049500e+10,4.589600e+10,1.411900e+10,-9.097600e+10,6.939100e+10,2019-12-31,71.173381
424,AAPL,2020,2020-09-30,2.716420e+11,9.700000e+09,1.105600e+10,7.734400e+10,5.741100e+10,3.801600e+10,4.061000e+09,...,6.533900e+10,1.697676e+10,2.168600e+11,7.309000e+09,-4.289000e+09,1.408100e+10,-8.682000e+10,8.067400e+10,2020-12-31,129.755631
425,AAPL,2021,2021-09-30,3.631720e+11,9.500000e+09,1.128400e+10,1.202330e+11,9.468000e+10,3.494000e+10,6.580000e+09,...,6.309000e+10,1.642679e+10,2.417830e+11,1.108500e+10,-1.454500e+10,1.446700e+10,-9.335300e+10,1.040380e+11,2021-12-31,174.713191
426,AAPL,2022,2022-09-30,3.913970e+11,8.700000e+09,1.110400e+10,1.305410e+11,9.980300e+10,2.364600e+10,4.946000e+09,...,5.067200e+10,1.594342e+10,2.332560e+11,1.070800e+10,-2.235400e+10,1.484100e+10,-1.107490e+11,1.221510e+11,2022-12-30,128.581643


In [ ]:
# Contar los tickers únicos por año
tickers_per_year = df_sp500.groupby("year")["ticker"].nunique()

# Ver el resultado
print(tickers_per_year)

year
2013    451
2014    463
2015    465
2016    471
2017    477
2018    485
2019    492
2020    494
2021    498
2022    501
2023    501
2024    107
2025      1
Name: ticker, dtype: int64


In [ ]:
# nos quedaremos con los registros con años inferiores o igual a 2023
df_sp500_11y = df_sp500.copy()
df_sp500_11y = df_sp500[df_sp500["year"] <= 2023]

In [ ]:
# contar los tickers únicos por año del nuevo dataframe
df_sp500_11y.groupby("year")["ticker"].nunique()

,ticker
year,
2013,451
2014,463
2015,465
2016,471
2017,477
2018,485
2019,492
2020,494
2021,498


In [ ]:
df_sp500_11y.nunique()

,0
ticker,501
year,11
fiscalDateEnding,276
totalRevenue,5150
depreciation,2434
depreciationAndAmortization,3586
ebitda,4889
netIncome,4743
cashAndCashEquivalentsAtCarryingValue,4759
inventory,4321


In [ ]:
df_sp500_11y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5377 entries, 0 to 5484
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ticker                                 5377 non-null   object 
 1   year                                   5377 non-null   int64  
 2   fiscalDateEnding                       5377 non-null   object 
 3   totalRevenue                           5238 non-null   float64
 4   depreciation                           3412 non-null   float64
 5   depreciationAndAmortization            5228 non-null   float64
 6   ebitda                                 5228 non-null   float64
 7   netIncome                              5238 non-null   float64
 8   cashAndCashEquivalentsAtCarryingValue  5352 non-null   float64
 9   inventory                              4919 non-null   float64
 10  currentNetReceivables                  4330 non-null   float64
 11  totalLiab

# ENRIQUECER DATOS CON EL SECTOR

In [ ]:
# descargar datos desde Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]

In [ ]:
sp500_table

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [ ]:
# me quedo con las columnas relevantes
sp500_sectors = sp500_table[['Symbol', 'GICS Sector']]
sp500_sectors.columns = ['ticker', 'sector']

In [ ]:
# merge de mi dataset con los sectores
df_merged = pd.merge(df_sp500_11y, sp500_sectors, how='left', on='ticker')

In [ ]:
df_merged['sector'].value_counts()

,count
sector,
Industrials,815
Financials,785
Information Technology,739
Health Care,645
Consumer Discretionary,557
Consumer Staples,394
Real Estate,341
Utilities,333
Materials,283


# GUARDADOS

In [ ]:
# Especifica la ruta en tu Google Drive
ruta_guardado = '/content/drive/MyDrive/ACADEMIC/MASTER DATA SCIENCE & AI (NUCLIO DIGITAL SCHOOL)/TFM/TFM SP500 COMPARTIDO/DATA_HIST/df_sp500_11y_decimalcoma.csv'

In [ ]:
# Guardar con separador decimal como coma (sin sectores)
df_sp500_11y.to_csv(ruta_guardado, index=False, sep=';', decimal=',')

**UTILIZAR ESTE FICHERO YA QUE CONTIENE LOS SECTORES!**

In [ ]:
# Especifica la ruta en tu Google Drive con sectores
ruta_guardado_con_sectores = '/content/drive/MyDrive/ACADEMIC/MASTER DATA SCIENCE & AI (NUCLIO DIGITAL SCHOOL)/TFM/TFM SP500 COMPARTIDO/DATA_HIST/df_sp500_11y_withsectors.csv'

In [ ]:
# Guardar con separador decimal como coma (con sectores)
df_merged.to_csv(ruta_guardado_con_sectores, index=False, sep=';', decimal=',')

In [ ]:
# se tiene que seguir con la parte de analisis y limpieza *ojo, este csv tiene los decimales como .*

pd.to_pickle(df_sp500_10y, "df_sp500_11y")
df_sp500_11y.to_csv("df_sp500_11y.csv")